In [17]:
import numpy as np

# np.random.seed(0)
num_objects = 10
num_stimuli = 4

class Shadlen:

	def __init__(self):
		self.actions_correctness = []
		self.step = 0
		self.criterion = 0.5
		self.stimuli = self.generate_stimuli(num_stimuli)
		self.state = self.to_one_hot([self.step] + self.stimuli)
	
	def to_one_hot(self, array, N=10):
		array = np.array(array).astype(int)
		labels = np.zeros((array.shape[0], N), dtype=np.float32)
		labels[np.arange(array.shape[0]), array] = 1.
		return labels
		
	def generate_stimuli(self, N_s):
		stimuli = []
		for i in range(N_s):
			stimuli.append(np.random.randint(0, num_objects))
		return stimuli
		
	def reset(self):
		self.__init__()
		return self.state
		
	def response(self, action):
		if self.step == 1:
			GT = float((np.sum(self.stimuli) / (num_objects * num_stimuli)) < self.criterion)
		else:
			GT = float((np.sum(self.stimuli) / (num_objects * num_stimuli)) > self.criterion)
			
		self.actions_correctness.append(GT == action)
		self.step += 1
		
		finished = False
		
		if self.step == 1:
			reward = 0.
			
		elif self.step == 2:
			if self.actions_correctness[-2]:
				reward = 0.
			else:
				finished = True
				if self.actions_correctness[-1]:
					reward = 1.2
				else:
					reward = -2.

		elif self.step == 3:		
			finished = True

			if self.actions_correctness[-2]:
				if self.actions_correctness[-1]:
					reward = 3.5
				else:
					reward = 0.5
			else:
				if self.actions_correctness[-1]:
					reward = 0.1
				else:
					reward = -1.					
		else:
			print("StepErr!")
			return

		self.stimuli = self.generate_stimuli(num_stimuli)
		self.state = self.to_one_hot([self.step] + self.stimuli)
		
		return reward, self.state, finished
	


In [18]:
s = Shadlen()

In [20]:
# import torch
import torch.nn as nn
from torch.autograd import Variable

a = s.reset()
# print(np.shape(a[]))
print(torch.tensor(a[1:5]).transpose(0,1))
inp = torch.tensor(a[1:5]).transpose(1,0)
inp2 = inp.view(len(inp), 4, -1)
print(len(inp))
print(inp2.size())
print(inp2)


NameError: name '_C' is not defined

In [11]:
a,b,c = s.response(1)
print(a)
print('hello')
print(b)
print('hello')
print(c)
if c == True:
    print ('heeey')

0.0
hello
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
hello
False


In [12]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)



NameError: name '_C' is not defined

In [38]:
print(model)

LSTM(
  (lstm): LSTM(50, 100)
  (linear): Linear(in_features=100, out_features=2, bias=True)
)


In [88]:
model = LSTM(input_dim = 50, hidden_dim = 10, batch_size=1, output_dim=2, num_layers=2)
generator = Shadlen()
gamma = 0.5
learning_rate = 0.05
eps = 1
R = []
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
finished = True
for i in range (200000):
    if (i%1000 == 0):
        eps = eps/2
    if finished:
        R.append(reward)
        state = generator.reset()
        inp = torch.tensor(state[0:5]).view(1,-1)
    else:
        state = next_state
        inp = next_inp
    Q = model.forward(inp)
    if Q[0]>Q[1]:
        action = 0
    else:
        action = 1
    if (torch.rand(1)<eps):
        action = 1 - action
    reward, next_state, finished = generator.response(action)
    
#         next_state = torch.tensor(next_state[1:5])
    next_inp = torch.tensor(next_state[0:5]).view(1,-1)
    next_Q = model.forward(next_inp)

    if finished:
        y = torch.tensor(reward)
    else:
        y = torch.tensor(reward) + gamma*torch.max(next_Q)
        
    Q_est = next_Q.detach().numpy()
    
    if (action == 0):
        Q_est[0] = y.detach().numpy()
    else:
        Q_est[1] = y.detach().numpy()
    
    Q_est = torch.tensor(Q_est)
#     Q_est = Q
#     Q_est[action] = y

#     loss = criterion(Q, Q_est)
    loss = criterion(y, torch.max(Q))
    loss.backward()
    optimizer.step()
    model.zero_grad()

import matplotlib.pylab as plt

plt.figure()
plt.plot(R)
plt.show()



RuntimeError: input.size(-1) must be equal to input_size. Expected 50, got 40

In [87]:
print(R)

[1.2, 1.2, -2.0, -2.0, 1.2, 0.5, 1.2, 0.5, -2.0, 1.2, -2.0, 3.5, 3.5, 3.5, 1.2, 0.5, 0.1, -1.0, 3.5, -2.0, 0.5, 0.1, 0.5, 1.2, -1.0, 3.5, -2.0, 0.5, 0.5, 0.1, -2.0, -2.0, 0.1, 0.1, -2.0, 0.1, 1.2, -1.0, 0.5, -2.0, -1.0, 0.1, -1.0, 0.1, -2.0, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, -2.0, 3.5, 3.5, 3.5, 1.2, 1.2, 0.5, 1.2, -2.0, -1.0, -2.0, -1.0, -2.0, 1.2, 1.2, 0.1, -2.0, -2.0, 0.1, -2.0, -2.0, -2.0, 0.1, 3.5, 1.2, 0.5, 0.1, -2.0, -1.0, 0.1, 1.2, -1.0, 0.1, -2.0, 1.2, 1.2, 0.5, -2.0, -2.0, -2.0, 0.1, -2.0, 0.5, 0.1, 1.2, -1.0, -2.0, 0.1, -1.0, 0.5, 1.2, -2.0, -2.0, 1.2, 0.1, -2.0, -2.0, 0.5, 0.5, 0.1, -2.0, -2.0, -2.0, 0.1, -2.0, 0.1, 1.2, 1.2, -1.0, -1.0, -2.0, 3.5, -2.0, 1.2, 0.1, 3.5, 0.1, -1.0, -2.0, -1.0, 1.2, 0.5, -1.0, 0.5, -2.0, 1.2, 1.2, 1.2, -1.0, -2.0, 1.2, 1.2, -1.0, 0.5, -1.0, -2.0, 0.5, 0.1, 3.5, -1.0, 1.2, 0.5, -1.0, -1.0, 1.2, 3.5, 0.5, 0.1, 3.5, 0.5, -2.0, -2.0, 3.5, -1.0, 3.5, 1.2, -2.0, 0.1, 1.2, -2.0, -1.0, -1.0, 1.2, 3.5, 0.5, -2.0, 1.2, 0.1, 0.5, -2.0, 0.5, 0.1, -2.0, 1.2, 1